In [123]:
from typing import Any, List
import pandas as pd
from collections import OrderedDict

from utils import print_spaced

pd.options.display.float_format = '{:.2f}'.format


We must consider our cash-flow position this quarter.
1. Whatever we end in this quarter will be what we will begin with in next. This amount will limit how much we can produce / limit our expansion initiatives / reduce our marketingor research investments.
2. There's a cap on how much loan we can get. If don't get much loan in next quarter and if we remain solely dependent on that - we will be done for good.

In [124]:
# Long-term debt capacity	
long_term_debt_cap = 5_005_345
# Total debt to date (loans in previous quarter + additions in other loans this quarter)	
used_loan = 2_100_000
# You are free to borrow up to an additional	
free_loan_cap = long_term_debt_cap - used_loan

print(f"Loan remaining to borrow: ${free_loan_cap:,}")

Loan remaining to borrow: $2,905,345


Our cash-flow needs depends on:
1. How much demand we want to serve in coming two quarters?
2. How much profits shall we predict in coming two quarters, if at all?

Note: Considering no or meager increment in non-variable expense avenues such as marketing, R&D etc.

In [125]:
sales_per_qtr_map = OrderedDict({
    "Quarter 1": 0,
    "Quarter 2": 0,
    "Quarter 3": 174,
    "Quarter 4": 534,
    "Quarter 5": 2_621,
    "Quarter 6": 2_712, # there isn't much jump here since we were limited by number of printers
})

qtr_sales_units = list(sales_per_qtr_map.values())


def get_header_to_unit_ration(df: pd.DataFrame, df_headers: List[str], sales_per_qtr: List[int]) -> List[float]:
    """
    Returns a list of ratios of sales per quarter to the unit sales in the given df.
    """
    header_sum_per_quarter = df.loc[df_headers, :].sum(axis=0)
    # print("header_sum_per_quarter: \n")
    # display(header_sum_per_quarter)

    ratio = []
    for i in range(len(sales_per_qtr)):
        if sales_per_qtr[i] == 0:
            ratio.append(0)
        else:
            ratio.append(round((header_sum_per_quarter[i]/sales_per_qtr[i]), 2))
    return ratio

def map_val_to_quarter(val: List[Any]):
    """
    Returns a dict mapping the given list of values to quarters.
    """
    return {f"qtr-{i+1}": val[i] for i in range(len(val))}

past_income_statements_df = pd.read_excel("sheets/qtr_7/past_income_statements.xlsx")
past_income_statements_df.set_index("Report Item", inplace=True)

marketing_exp_headers = ["+ Advertising", "+ Internet Marketing Expenses"]
sales_exp_headers = ["+ Sales Force Expense", "+ Store and Web Sales Center Expenses", "+ Shipping", "- Rebates", "+ Quality Costs"]
manufacturing_exp_headers = ["- Cost of Goods Sold", "+ Excess Capacity Cost"]
rnd_and_improvement_exp_header = ["Research and Development", "+ System Improvement Costs", "+ Web Sales Productivity Expenses"]
gross_profit_header = ["Gross Profit"]

print_spaced("quarter_unit_sales: ", qtr_sales_units)

# ----------------------------------------------- marketing -----------------------------------------------
marketing_exp_to_unit_ratio = get_header_to_unit_ration(past_income_statements_df, marketing_exp_headers, qtr_sales_units)
print_spaced("marketing_exp_to_unit_ratio", map_val_to_quarter(marketing_exp_to_unit_ratio))

# ----------------------------------------------- sales -----------------------------------------------
sales_exp_to_unit_ratio = get_header_to_unit_ration(past_income_statements_df, sales_exp_headers, qtr_sales_units)
print_spaced("sales_exp_to_unit_ratio", map_val_to_quarter(sales_exp_to_unit_ratio))

# ----------------------------------------------- manufacturing -----------------------------------------------
manufacturing_exp_to_unit_ratio = get_header_to_unit_ration(past_income_statements_df, manufacturing_exp_headers, qtr_sales_units)
print_spaced("manufacturing_exp_to_unit_ratio", map_val_to_quarter(manufacturing_exp_to_unit_ratio))


# ----------------------------------------------- rnd_and_improvement -----------------------------------------------
rnd_and_improvement_exp_to_unit_ratio = get_header_to_unit_ration(past_income_statements_df, rnd_and_improvement_exp_header, qtr_sales_units)
print_spaced("rnd_and_improvement_exp_to_unit_ratio", map_val_to_quarter(rnd_and_improvement_exp_to_unit_ratio))

# ----------------------------------------------- gross_profit -----------------------------------------------
gross_profit_to_unit_ratio = get_header_to_unit_ration(past_income_statements_df, gross_profit_header, qtr_sales_units)
print_spaced("gross_profit_to_unit_ratio", map_val_to_quarter(gross_profit_to_unit_ratio))


quarter_unit_sales: 
[0, 0, 174, 534, 2621, 2712]



marketing_exp_to_unit_ratio
{'qtr-1': 0, 'qtr-2': 0, 'qtr-3': 282.09, 'qtr-4': 282.1, 'qtr-5': 202.24, 'qtr-6': 123.95}



sales_exp_to_unit_ratio
{'qtr-1': 0, 'qtr-2': 0, 'qtr-3': 2968.21, 'qtr-4': 1756.37, 'qtr-5': 488.0, 'qtr-6': 458.04}



manufacturing_exp_to_unit_ratio
{'qtr-1': 0, 'qtr-2': 0, 'qtr-3': 736.97, 'qtr-4': 582.98, 'qtr-5': 283.15, 'qtr-6': 458.0}



rnd_and_improvement_exp_to_unit_ratio
{'qtr-1': 0, 'qtr-2': 0, 'qtr-3': 339.08, 'qtr-4': 224.72, 'qtr-5': 317.63, 'qtr-6': 177.67}



gross_profit_to_unit_ratio
{'qtr-1': 0, 'qtr-2': 0, 'qtr-3': 582.86, 'qtr-4': 543.24, 'qtr-5': 292.13, 'qtr-6': 519.99}




This is the manufacturing expense ratio for past quarters:

```
manufacturing_exp_to_unit_ratio
{'qtr-1': 0, 'qtr-2': 0, 'qtr-3': 736.97, 'qtr-4': 582.98, 'qtr-5': 283.15, 'qtr-6': 458.0}
```

We see a sharp jump in qtr-6.

But overall, it seems fine, since its followed by a sharp jump in gross-margin as well.

```
gross_profit_to_unit_ratio
{'qtr-1': 0, 'qtr-2': 0, 'qtr-3': 582.86, 'qtr-4': 543.24, 'qtr-5': 292.13, 'qtr-6': 519.99}
```


In [126]:
# find estimate quality cost for n number of units using linear regression model.
from sklearn.linear_model import LinearRegression
import numpy as np

# find estimate quality cost for n number of units using linear regression model.
def get_model_for_sales(df: pd. DataFrame, column: List[str], sales_array: List[int]) -> LinearRegression:
    """
    Returns a LinearRegression model trained on the given df and column.
    Ex: pass it a sales figure and give it a column to train on.
    It will predict how the column will change with the given sales figure.
    Will be helpful in determining expense in a particular header for a given sales figure.
    """
    model = LinearRegression()
    X = np.array(sales_array).reshape(-1,1) 
    Y = np.array(df.loc[column, :].values[0])
    model.fit(X, Y)
    return model

quality_model = get_model_for_sales(past_income_statements_df, ["+ Quality Costs"], qtr_sales_units)
quality_cost_for_10000 = quality_model.predict(np.array([[10000]]))[0]

print_spaced("quality_cost_for_10000", quality_cost_for_10000)



quality_cost_for_10000
454376.44992651




In [127]:



def predict_costs(anticipated_demand: int, past_income_statements_df: pd.DataFrame, expense_columns: List[str], qtr_sales_units: List[int]):
    costs = []
    for column in expense_columns:
        model = get_model_for_sales(past_income_statements_df, [column], qtr_sales_units)
        cost = model.predict(np.array([[anticipated_demand]]))[0]
        costs.append(round(cost, 2))
    return costs

cost_columns_to_predict = ["+ Shipping", "+ Depreciation", "+ Quality Costs"]
various_demands_array = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
demand_vs_cost_df = pd.DataFrame(columns=cost_columns_to_predict, index=various_demands_array)

for demand in various_demands_array:
    costs = predict_costs(demand, past_income_statements_df, cost_columns_to_predict, qtr_sales_units)
    demand_vs_cost_df.loc[demand, :] = costs

display(demand_vs_cost_df.style.format('{:,}'))

    

,+ Shipping,+ Depreciation,+ Quality Costs
1000,"32,776.17","37,491.2","56,414.74"
2000,"60,015.37","62,891.39","100,632.71"
3000,"87,254.56","88,291.58","144,850.68"
4000,"114,493.75","113,691.77","189,068.65"
5000,"141,732.94","139,091.96","233,286.61"
6000,"168,972.13","164,492.15","277,504.58"
7000,"196,211.32","189,892.34","321,722.55"
8000,"223,450.51","215,292.53","365,940.52"
9000,"250,689.7","240,692.72","410,158.48"
10000,"277,928.89","266,092.91","454,376.45"


In [128]:
# add COGS to the demand_vs_cost_df
# use scale of economies for predicting COGS

noram_cogs = pd.read_excel("sheets/qtr_7/CostOfProductionRegional-Q7.xlsx", sheet_name="Cost of Production Estimate - N").set_index("Brand")
europe_cogs = pd.read_excel("sheets/qtr_7/CostOfProductionRegional-Q7.xlsx", sheet_name="Cost of Production Estimate - E").set_index("Brand")
apac_cogs = pd.read_excel("sheets/qtr_7/CostOfProductionRegional-Q7.xlsx", sheet_name="Cost of Production Estimate - A").set_index("Brand")

average_cogs_df =  pd.DataFrame(columns=noram_cogs.columns, index=noram_cogs.index)

for demand in noram_cogs.index:
    average_cogs_df.loc[demand, :] = (noram_cogs.loc[demand, :] + europe_cogs.loc[demand, :] + apac_cogs.loc[demand, :])/3

average_cogs_df = average_cogs_df.round(2)
print("average_cogs_df: \n")
display(average_cogs_df)

average_cogs_df: 



,100 units,250 units,500 units,"1,000 units","5,000 units"
Brand,,,,,
2. Velo Pro,765.33,550.93,414.51,345.04,278.60
Velo Nature Max,801.76,587.23,450.60,380.70,310.94
Velo Terrain XL,1066.46,850.84,712.35,638.74,540.42
Velo Joy,934.62,719.58,582.07,510.41,427.05
Velo Pro Adv,827.35,612.74,475.96,405.76,333.65
Velo Joy Adv,940.53,725.47,587.93,516.21,432.36
Velo Pro Plus,800.77,586.25,449.62,379.72,309.97


In [129]:
all_brands_avg = average_cogs_df.mean(axis=0)
display(all_brands_avg)

economy_of_scale_breakpoints = [100, 250, 500, 1000, 5000]
scale_pricing = dict(zip(economy_of_scale_breakpoints, list(all_brands_avg.values)))


100 units     876.69
250 units     661.86
500 units     524.72
1,000 units   453.80
5,000 units   376.14
dtype: object

In [142]:
# add cogs to demand_vs_cost_df
demand_vs_cost_df

def choose_higher_range(demand: int, breakpoints: List[int]) -> int:
    """
    Finds where the demand lies in the given breakpoints and returns the higher range.
    """
    for bp in breakpoints:
        if demand <= bp:
            return bp
    return breakpoints[-1]

for demand in demand_vs_cost_df.index:
    for bp in economy_of_scale_breakpoints:
        if demand >= bp:
            pricing_for_bp = scale_pricing[choose_higher_range(demand, economy_of_scale_breakpoints)]
            print(f"Scale pricing for demand {demand} is {pricing_for_bp}")
            demand_vs_cost_df.loc[demand, "+ Cost of Goods Sold"] = pricing_for_bp * demand
            break

# add a total column
demand_vs_cost_df.loc[:, "Total"] = demand_vs_cost_df.sum(axis=1)

display(demand_vs_cost_df.round(2))

Scale pricing for demand 1000 is 453.7966858750777
Scale pricing for demand 2000 is 376.1424993667924
Scale pricing for demand 3000 is 376.1424993667924
Scale pricing for demand 4000 is 376.1424993667924
Scale pricing for demand 5000 is 376.1424993667924
Scale pricing for demand 6000 is 376.1424993667924
Scale pricing for demand 7000 is 376.1424993667924
Scale pricing for demand 8000 is 376.1424993667924
Scale pricing for demand 9000 is 376.1424993667924
Scale pricing for demand 10000 is 376.1424993667924


,+ Shipping,+ Depreciation,+ Quality Costs,+ Cost of Goods Sold,Total
1000,32776.17,37491.20,56414.74,453796.69,2321915.18
2000,60015.37,62891.39,100632.71,752285.00,3903297.87
3000,87254.56,88291.58,144850.68,1128427.50,5795297.27
4000,114493.75,113691.77,189068.65,1504570.00,7687296.67
5000,141732.94,139091.96,233286.61,1880712.50,9579296.03
6000,168972.13,164492.15,277504.58,2256855.00,11471295.42
7000,196211.32,189892.34,321722.55,2632997.50,13363294.82
8000,223450.51,215292.53,365940.52,3009139.99,15255294.22
9000,250689.70,240692.72,410158.48,3385282.49,17147293.58
10000,277928.89,266092.91,454376.45,3761424.99,19039292.97
